In [1]:
from drugs import Drugs
import pandas as pd

In [2]:
df = pd.read_csv("../data/drugs_train.csv")

In [3]:
df_ingredient= pd.read_csv("../data/active_ingredients.csv")

In [4]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, random_state=40, test_size=0.2)
ingredient_train = df_ingredient[df_ingredient.drug_id.isin(df_train.drug_id)]
ingredient_test = df_ingredient[df_ingredient.drug_id.isin(df_test.drug_id)]

In [5]:
df_train.drug_id.nunique()

6851

In [6]:
ingredient_train.drug_id.nunique()

6851

In [7]:
df_full_train = df_train.merge(ingredient_train)

In [8]:
df_full_test = df_test.merge(ingredient_test)

In [15]:
drugs = Drugs()

In [16]:
pipe = drugs.processing_pipe

In [17]:
pipe.fit(df_full_train)

Pipeline(steps=[('text_cleaner',
                 TextCleaner(columns=['description', 'administrative_status',
                                      'marketing_status', 'dosage_form',
                                      'marketing_authorization_status',
                                      'marketing_authorization_process',
                                      'pharmaceutical_companies'])),
                ('date_cleaner',
                 DateCleaner(columns=['marketing_declaration_date',
                                      'marketing_authorization_date'])),
                ('percentage_encoder', Percent...
                 DropColumnsCleaner(columns=['drug_id', 'description',
                                             'administrative_status',
                                             'marketing_status',
                                             'approved_for_hospital_use',
                                             'reimbursement_rate',
                               

In [20]:
pipe.fit(df_full_train)

Pipeline(steps=[('text_cleaner',
                 TextCleaner(columns=['description', 'administrative_status',
                                      'marketing_status', 'dosage_form',
                                      'marketing_authorization_status',
                                      'marketing_authorization_process',
                                      'pharmaceutical_companies'])),
                ('date_cleaner',
                 DateCleaner(columns=['marketing_declaration_date',
                                      'marketing_authorization_date'])),
                ('percentage_encoder', Percent...
                 DropColumnsCleaner(columns=['drug_id', 'description',
                                             'administrative_status',
                                             'marketing_status',
                                             'approved_for_hospital_use',
                                             'reimbursement_rate',
                               

In [21]:
x = pipe.transform(df_full_train)

In [22]:
x.shape

(6851, 8)

In [14]:
drugs.fit(df_full_train)

XGBoostError: [00:56:30] ../src/data/data.cc:592: Check failed: labels_.Size() == num_row_ (8847 vs. 5945) : Size of labels must equal to number of rows.
Stack trace:
  [bt] (0) /home/abdelelyds/.local/lib/python3.8/site-packages/xgboost/lib/libxgboost.so(+0x112d29) [0x7efc97856d29]
  [bt] (1) /home/abdelelyds/.local/lib/python3.8/site-packages/xgboost/lib/libxgboost.so(+0x11edc9) [0x7efc97862dc9]
  [bt] (2) /home/abdelelyds/.local/lib/python3.8/site-packages/xgboost/lib/libxgboost.so(+0x1a9209) [0x7efc978ed209]
  [bt] (3) /home/abdelelyds/.local/lib/python3.8/site-packages/xgboost/lib/libxgboost.so(+0x1b33d1) [0x7efc978f73d1]
  [bt] (4) /home/abdelelyds/.local/lib/python3.8/site-packages/xgboost/lib/libxgboost.so(XGBoosterUpdateOneIter+0x68) [0x7efc977dd478]
  [bt] (5) /lib/x86_64-linux-gnu/libffi.so.7(+0x6ff5) [0x7efd12b84ff5]
  [bt] (6) /lib/x86_64-linux-gnu/libffi.so.7(+0x640a) [0x7efd12b8440a]
  [bt] (7) /usr/lib/python3.8/lib-dynload/_ctypes.cpython-38-x86_64-linux-gnu.so(_ctypes